# Jinja темплейты

Jinja2 — самый популярный шаблонизатор в языке программирования Python.

Синтаксис Jinja2 сильно похож на Django-шаблонизатор, но при этом дает возможность использовать чистые Python выражения и поддерживает гибкую систему расширений.
Ниже приведем основные примеры того, как использовать данные шаблоны.

In [1]:
from jinja2 import Template

In [21]:
# Обычные выражения
Template("{{ 10 + 3 }}").render()

'13'

In [3]:
# Вывод переменных
Template("{{ var }}").render(var=12)

'12'

In [4]:
class Foo:
    def __str__(self):
        return "This is an instance of Foo class"
Template("{{ var }}").render(var=Foo())

'This is an instance of Foo class'

In [5]:
# Операторы в Jinja2 % функция %

text = '{% for item in range(5) %}Hello {{ name }}! {% endfor %}'
template = Template(text)
print(template.render(name=u'Вася'))

Hello Вася! Hello Вася! Hello Вася! Hello Вася! Hello Вася! 


In [6]:
# Объявить переменные
template = Template("{% set a, b, c = 'foo', 'фуу', 'föö' %}")
m = template.module
print(m.a)
print(m.b)
print(m.c)

foo
фуу
föö


Переменные определяются для хранения результатов сложных операций, так чтобы их можно было использовать дальше в шаблоне. Переменные, определенные вне управляющих конструкций (о них дальше), ведут себя как глобальные переменные и доступны внутри любой структуры. Тем не менее переменные, созданные внутри конструкций, ведут себя как локальные переменные и видимы только внутри этих конкретных конструкций. Единственное исключение — инструкция if.

In [9]:
template = Template('{% if bookmarks %} <p>User has some bookmarks</p> {% endif %}')
template.render()

''

Если значение переменной bookmarks – True, тогда будет выведена строка
```<p>User has some bookmarks</p>```.

Стоит запомнить, что в Jinja, если у переменной нет значения, она возвращает False.

In [11]:
Template('{{ "User is logged in" if loggedin else "User is not logged in" }}').render()

'User is not logged in'

## Фильтры

Фильтры изменяют переменные до процесса рендеринга. Синтаксис использования фильтров следующий:

In [ ]:
Template("{{ comment|title }}")

Фильтр title делает заглавной первую букву в каждом слове. Если значение переменной comment — "dust in the wind", то вывод будет "Dust In The Wind".

Ниже представлены широко используемые фильтры:

| Название	| Описание |
| --- | --- |
| upper	| делает все символы заглавными |
| lower	| приводит все символы к нижнему регистру |
| capitalize	| делает заглавной первую букву и приводит остальные к нижнему регистру |
| escape	| экранирует значение |
| safe	| предотвращает экранирование |
| length	| возвращает количество элементов в последовательности |
| trim	| удаляет пустые символы в начале и в конце |
| random	| возвращает случайный элемент последовательности |

## Макросы

Макросы - это по сути те же функции в Python, только в Jinja2.

Суть в том, чтобы сделать код, который можно использовать повторно, просто присвоив ему название.

In [13]:
template = """
{% macro render_posts(post_list, sep=False) %}
    <div>
        {% for post in post_list %}
            <h2>{{ post.title }}</h2>
            <article>
                {{ post.html|safe }}
            </article>
        {% endfor %}
        {% if sep %}<hr>{% endif %}
    </div>
{% endmacro %}
"""

В этом примере создан макрос render_posts, который принимает обязательный аргумент post_list и необязательный аргумент sep. Использовать его нужно следующим образом:

In [14]:
Template("""{{ render_posts(posts) }}""")

<Template memory:7fefc11c9880>

In [15]:
# Можно импортировать макросы из отдельного файла
"""{% import "macros.html" as  macros %}"""

'{% import "macros.html" as  macros %}'

## Экранирование

Jinja по умолчанию автоматически экранирует вывод переменной в целях безопасности. Поэтому если переменная содержит, например, такой HTML-код:
```<p>Escaping in Jinja</p>```, он отрендерится в виде ```&lt;p&gt;Escaping in Jinja&lt;/p&gt;```. Благодаря этому HTML-коды будут отображаться в браузере, а не интерпретироваться. Если есть уверенность, что данные безопасны и их точно можно рендерить, стоит воспользоваться фильтром safe. Например:

In [ ]:
"""{% set html = "<p>Escaping in Jinja</p>" %}"""
"""{{ html|safe }}"""

# Вывод:
"""<p>Escaping in Jinja</p>"""

Использовать фильтр safe в большом блоке кода будет неудобно, поэтому в Jinja есть оператор autoescape, который используется, чтобы отключить экранирование для большого объема данных. Он может принимать аргументы true или false для включения и отключения экранирования, соответственно. Например:

In [17]:
tempate = """
{% autoescape true %}
    Escaping enabled
{% endautoescape %}
"""

## Вложенные шаблоны

Инструкция include рендерит шаблон внутри другого шаблона. Она широко используется, чтобы рендерить статический раздел, который повторяется в разных местах сайта. Вот синтаксис include:

Предположим, что навигационное меню хранится в файле nav.html, сохраненном в папке templates.

Чтобы добавить это меню в home.html, нужно использовать следующий код:

In [18]:
# Внутри <body> </body>
template_inside_body = """{% include 'nav.html' %}"""

## Наследование шаблонов

Наследование шаблонов — один из самых мощных элементов шаблонизатора Jinja. Его принцип похож на ООП (объектно-ориентированное программирование). Все начинается с создания базового шаблона, который содержит в себе скелет HTML и отдельные маркеры, которые дочерние шаблоны смогут переопределять. Маркеры создаются с помощью инструкции ```block```. Дочерние шаблоны используют инструкцию ```extends``` для наследования или расширения основного шаблона. Вот пример:

In [19]:
base_template = """
{# Это шаблон templates/base.html #}
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{% block title %}Default Title{% endblock %}</title>
</head>
<body>

    {% block nav %}
        <ul>
            <li><a href="/home">Home</a></li>
            <li><a href="/api">API</a></li>
        </ul>
    {% endblock %}

    {% block content %}

    {% endblock %}
</body>
</html>
"""


Это базовый шаблон ```base.html```. Он создает три блока с помощью ```block```, которые впоследствии будут заполнены дочерними шаблонами. Инструкция ```block``` принимает один аргумент — название блока. Внутри шаблона это название должно быть уникальным, иначе возникнет ошибка.

Дочерний шаблон — это шаблон, который растягивает базовый шаблон. Он может добавлять, перезаписывать или оставлять элементы родительского блока. Вот как можно создать дочерний шаблон.

In [ ]:
child_template = """
{# Это шаблон templates/child.html #}
{% extends 'base.html' %}


{% block content %}
    {% for bookmark in bookmarks %}
        <p>{{ bookmark.title }}</p>
    {% endfor %}
{% endblock %}
"""

Инструкция extends сообщает Jinja, что child.html — это дочерний элемент, наследник base.html. Когда Jinja обнаруживает инструкцию extends, он загружает базовый шаблон, то есть base.html, а затем заменяет блоки контента внутри родительского шаблона блоками с теми же именами из дочерних шаблонов. Если блок с соответствующим названием не найден, используется блок родительского шаблона.

Видим, что в этом случае перезаписывается только блок ```content```.

После перезаписи блока на контент из родительского шаблона все еще можно ссылаться с помощью функции super(). Обычно она используется, когда в дополнение к контенту дочернего шаблона нужно добавить содержимое из родительского. Например:

In [20]:
template_child = """{# Это шаблон templates/child.html #}
{% extends 'base.html' %}

{% block title %}
    Child Title
{% endblock %}

{% block nav %}
    {{ super() }} {# referring to the content in the parent templates #}
    <li><a href="/contact">Contact</a></li>
    <li><a href="/career">Career</a></li>
{% endblock %}

{% block content %}
    {% for bookmark in bookmarks %}
        <p>{{ bookmark.title }}</p>
    {% endfor %}
{% endblock %}"""